In [ ]:
import os
import cv2
import numpy as np

#Encoding and Split data into Train/Test Sets
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

#Tensorflow Keras CNN Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation, Conv2D, MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam, SGD, Adagrad,Adadelta,RMSprop

#Plot Images
import matplotlib.pyplot as plt

folder_dir = '/content/drive/MyDrive/Image_CLF_Datasets/flowers'

In [ ]:
data = []
label = []

#crop the image to 128 x 128
SIZE = 128

for folder in os.listdir(folder_dir):
    for file in os.listdir(os.path.join(folder_dir, folder)):
        if file.endswith("jpg"):
            label.append(folder)
            img = cv2.imread(os.path.join(folder_dir, folder, file))
            img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            im = cv2.resize(img_rgb, (SIZE,SIZE))
            data.append(im)
        else:
            continue

In [ ]:
# Convert data into numerical values
data_arr = np.array(data)
label_arr = np.array(label)

In [ ]:
# print(data_arr)

In [ ]:
# print(label_arr)

In [ ]:
# Use label encoder and normalize the data
encoder = LabelEncoder()
y = encoder.fit_transform(label_arr)
print(y)
y = to_categorical(y,5)
print(y)
X = data_arr/255

[4 4 4 ... 2 2 2]
[[0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 [0. 0. 0. 0. 1.]
 ...
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]
 [0. 0. 1. 0. 0.]]


In [ ]:
#split the dataset into 80% training and 20% testing set
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.30, random_state=10)

In [ ]:
# Build Neura Network for flower classification:

model = Sequential()
model.add(Conv2D(filters=64,kernel_size=(3,3),padding='Same',activation='relu', input_shape = (SIZE, SIZE,3) ))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "Same", activation = 'relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "Same", activation = 'relu'))
model.add(Conv2D(filters = 128, kernel_size = (3,3), padding = "Same", activation = 'relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dropout(rate=0.25))
model.add(Dense(5, activation='softmax'))

In [ ]:
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 128, 128, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 64, 64, 64)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 64, 64, 128)       73856     
                                                                 
 conv2d_2 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 conv2d_3 (Conv2D)           (None, 64, 64, 128)       147584    
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 32, 32, 128)      0         
 2D)                                                    

In [ ]:
# We need to create more training images to prevent overfitting before compiling

datagen = ImageDataGenerator(
    rotation_range = 20,
    zoom_range = 0.20,
    width_shift_range = 0.3,
    height_shift_range = 0.3,
    horizontal_flip = True,
    vertical_flip = True
)

datagen.fit(X_train)

In [ ]:
#compile the model

model.compile(optimizer=Adam(learning_rate=0.0001),loss='categorical_crossentropy',metrics=['accuracy'])
batch_size = 32
epochs = 64
history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),
                              epochs = epochs,
                              validation_data = (X_test,y_test),
                              verbose = 2)

<ipython-input-11-9f23205af313>:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(datagen.flow(X_train,y_train, batch_size=batch_size),


Epoch 1/64
95/95 - 33s - loss: 1.4864 - accuracy: 0.3337 - val_loss: 1.3588 - val_accuracy: 0.4275 - 33s/epoch - 350ms/step
Epoch 2/64
95/95 - 18s - loss: 1.3073 - accuracy: 0.4270 - val_loss: 1.1502 - val_accuracy: 0.5008 - 18s/epoch - 190ms/step
Epoch 3/64
95/95 - 17s - loss: 1.2149 - accuracy: 0.4912 - val_loss: 1.1588 - val_accuracy: 0.4923 - 17s/epoch - 178ms/step
Epoch 4/64
95/95 - 22s - loss: 1.1708 - accuracy: 0.5263 - val_loss: 1.0134 - val_accuracy: 0.6065 - 22s/epoch - 236ms/step
Epoch 5/64
95/95 - 16s - loss: 1.1398 - accuracy: 0.5366 - val_loss: 1.0349 - val_accuracy: 0.5895 - 16s/epoch - 173ms/step
Epoch 6/64
95/95 - 16s - loss: 1.0724 - accuracy: 0.5776 - val_loss: 0.9438 - val_accuracy: 0.6335 - 16s/epoch - 172ms/step
Epoch 7/64
95/95 - 16s - loss: 1.0730 - accuracy: 0.5796 - val_loss: 0.9443 - val_accuracy: 0.6397 - 16s/epoch - 170ms/step
Epoch 8/64
95/95 - 17s - loss: 1.0426 - accuracy: 0.5892 - val_loss: 0.9306 - val_accuracy: 0.6343 - 17s/epoch - 183ms/step
Epoch 9/

In [ ]:
# let model identify the flower

categories = np.sort(os.listdir(folder_dir))
fig, ax = plt.subplots(6,6, figsize=(25,40))

for i in range(6):
    for j in range(6):
        k = int(np.random.random_sample() * len(X_test))
        if(categories[np.argmax(y_test[k])] == categories[np.argmax(model.predict(X_test)[k])]):
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='green')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])],color='green')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE,SIZE,3), cmap='gray')
        else:
            ax[i,j].set_title("TRUE: " + categories[np.argmax(y_test[k])], color='red')
            ax[i,j].set_xlabel("PREDICTED: " + categories[np.argmax(model.predict(X_test)[k])],color='red')
            ax[i,j].imshow(np.array(X_test)[k].reshape(SIZE,SIZE,3), cmap='gray')

41/41 [==============================] - 1s 19ms/step
